In [1]:
import chess 
import chess.pgn
import numpy as np
from random import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange
from sklearn.model_selection import train_test_split

# Data Loader

In [2]:
class GameLoader():
    def __init__(self):
        pass
    def labeling_Win_Loss(self,file_pth):
        W_Win=[1]
        W_label=[]
        W_win,B_win=self.win_loss_classifier(file_pth)
        for i in trange(len(W_win)):
            W_label.append(W_Win)
        B_Win=[0]
        B_label=[]
        for i in trange(len(B_win)):
            B_label.append(B_Win)
        return W_win,W_label,B_win,B_label
    
    def convert_to_tensor(self,W_win,B_win,W_label,B_label,device):
        W_Win_tensor=torch.tensor(W_win,dtype=torch.float32,device=device)
        W_label_tensor=torch.tensor(W_label,dtype=torch.float32,device=device)
        B_win_tensor=torch.tensor(B_win,dtype=torch.float32,device=device)
        B_label_tensor=torch.tensor(B_label,dtype=torch.float32,device=device)
        return W_win_tensor,W_label_tensor,B_win_tensor,B_label_tensor
    
    def win_loss_classifier(self,file_pth):
        pgn_file = open(file_pth)
        W_win = []
        B_win= []
        i = 0
        for i in trange(20000):
            if i > 0:
                game = chess.pgn.read_game(pgn_file)
                if game.headers["Result"] == "1-0":
                    temp1 = self.randPosiGeneratorFromGame(game)
                    W_win+= temp1
                elif game.headers["Result"] == "0-1":
                    temp2 = self.randPosiGeneratorFromGame(game)
                    B_win += temp2
                if game is None:
                    break
            i = i + 1
        pgn_file.close()
        return W_win,B_win
        
    def randPosiGeneratorFromGame(self,game):
        random_positions_array = []
        positions = []
        board = game.board()

        for move in game.mainline_moves():
            if not board.is_capture(move):
                position = board.fen()
                bitstring_position = self.fen_to_bitstring(position)
                positions.append(bitstring_position)

            board.push(move)
            no_capture_posis = len(positions)
        indices = list(range(5, no_capture_posis))
        shuffle(indices)
        selected_indices = indices[:15]
        random_positions_array = [positions[index] for index in selected_indices]
        return random_positions_array

    def fen_to_bitstring(self,fen):
        board = chess.Board(fen)
        bitboard = np.zeros(64*6*2+5)
        piece_idx = {'p': 0, 'n': 1, 'b': 2, 'r': 3, 'q': 4, 'k': 5}
        for i in range(64):
            if board.piece_at(i):
                color = int(board.piece_at(i).color) + 1
                bitboard[(piece_idx[board.piece_at(i).symbol().lower()] + i * 6) * color] = 1
        bitboard[-1] = int(board.turn)
        bitboard[-2] = int(board.has_kingside_castling_rights(True))
        bitboard[-3] = int(board.has_kingside_castling_rights(False))
        bitboard[-4] = int(board.has_queenside_castling_rights(True))
        bitboard[-5] = int(board.has_queenside_castling_rights(False))
        return bitboard

In [3]:
Game_Load=GameLoader()
W_win,W_label,B_win,B_label = Game_Load.labeling_Win_Loss("CCRL-404.[881513].pgn")

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/110670 [00:00<?, ?it/s]

  0%|          | 0/87269 [00:00<?, ?it/s]

# Getting Weights

In [4]:
x=20000
White_win=W_win[0:x]
Black_win=B_win[0:x]
White_label=W_label[0:x]
Black_label=B_label[0:x]

In [5]:
np.shape(White_win)

(20000, 773)

In [6]:
White_win_tensor=torch.tensor(White_win,dtype=torch.float32)
Black_win_tensor=torch.tensor(Black_win,dtype=torch.float32)

C:\Users\MR_X\AppData\Local\Temp\ipykernel_2556\1846430892.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  White_win_tensor=torch.tensor(White_win,dtype=torch.float32)


In [7]:
encoder_data=torch.concatenate((White_win_tensor,Black_win_tensor),axis=0)

In [8]:
len(encoder_data)

40000

In [9]:
class Autoencoders(nn.Module):
    def __init__(self):
        super(Autoencoders,self).__init__()
        self.Layer1encoder=nn.Sequential(
            nn.Linear(773,600),
            nn.ReLU(),
            nn.Linear(600,773)
        )
        self.Layer2encoder=nn.Sequential(
            nn.Linear(773,600),
            nn.ReLU(),
            nn.Linear(600,400),
            nn.ReLU(),
            nn.Linear(400,600),
            nn.ReLU(),
            nn.Linear(600,773)
        )
        self.Layer3encoder=nn.Sequential(
            nn.Linear(773,600),
            nn.ReLU(),
            nn.Linear(600,400),
            nn.ReLU(),
            nn.Linear(400,200),
            nn.ReLU(),
            nn.Linear(200,400),
            nn.ReLU(),
            nn.Linear(400,600),
            nn.ReLU(),
            nn.Linear(600,773)
        )
        self.Layer4encoder=nn.Sequential(
            nn.Linear(773,600),
            nn.ReLU(),
            nn.Linear(600,400),
            nn.ReLU(),
            nn.Linear(400,200),
            nn.ReLU(),
            nn.Linear(200,100),
            nn.ReLU(),
            nn.Linear(100,200),
            nn.ReLU(),
            nn.Linear(200,400),
            nn.ReLU(),
            nn.Linear(400,600),
            nn.ReLU(),
            nn.Linear(600,773)
        )
    def GetWeights(self,input_data):
        Weights=[]
        loss=nn.MSELoss()
        num_epochs=200
        #1st layer
        optimizer1=optim.SGD(self.Layer1encoder.parameters(),lr=0.001)
        for epoch in trange(num_epochs):
            output1=self.Layer1encoder(input_data)
            loss1=loss(output1,input_data)
            optimizer1.zero_grad()
            loss1.backward()
            optimizer1.step()
        print(loss1)
        Weight1=list(self.Layer1encoder.parameters())[0]
        print(np.shape(Weight1))
        Weights.append(Weight1)
        
        #2nd layer
        optimizer2=optim.SGD(self.Layer2encoder.parameters(),lr=0.001)
        for epoch in trange(num_epochs):
            output2=self.Layer2encoder(input_data)
            loss2=loss(output2,input_data)
            optimizer2.zero_grad()
            loss2.backward()
            optimizer2.step()
        print(loss2)
        Weight2=list(self.Layer2encoder.parameters())[2]
        print(np.shape(Weight2))
        Weights.append(Weight2) 
        
        #3rd layer
        optimizer3=optim.SGD(self.Layer3encoder.parameters(),lr=0.001)
        for epoch in trange(num_epochs):
            output3=self.Layer3encoder(input_data)
            loss3=loss(output3,input_data)
            optimizer3.zero_grad()
            loss3.backward()
            optimizer3.step()
        print(loss3)
        Weight3=list(self.Layer3encoder.parameters())[4]
        print(np.shape(Weight3))
        Weights.append(Weight3) 
        
        #4th layer
        optimizer4=optim.SGD(self.Layer4encoder.parameters(),lr=0.001)
        for epoch in trange(num_epochs):
            output4=self.Layer4encoder(input_data)
            loss4=loss(output4,input_data)
            optimizer4.zero_grad()
            loss4.backward()
            optimizer4.step()
        print(loss4)
        Weight4=list(self.Layer4encoder.parameters())[6]
        print(np.shape(Weight4))
        Weights.append(Weight4) 
        
        return Weights

In [10]:
getWeights=Autoencoders()
Weights=getWeights.GetWeights(encoder_data)

  0%|          | 0/200 [00:00<?, ?it/s]

tensor(0.0299, grad_fn=<MseLossBackward0>)
torch.Size([600, 773])


  0%|          | 0/200 [00:00<?, ?it/s]

tensor(0.0290, grad_fn=<MseLossBackward0>)
torch.Size([400, 600])


  0%|          | 0/200 [00:00<?, ?it/s]

tensor(0.0287, grad_fn=<MseLossBackward0>)
torch.Size([200, 400])


  0%|          | 0/200 [00:00<?, ?it/s]

tensor(0.0287, grad_fn=<MseLossBackward0>)
torch.Size([100, 200])


# Model

In [11]:
class Pos2Vec(nn.Module):
    def __init__(self):
        super(Pos2Vec, self).__init__()
        self.weights1 = Weights[0]
        self.weights2 = Weights[1]
        self.weights3 = Weights[2]
        self.weights4 = Weights[3]
        self.encoder = nn.Sequential(
            nn.Linear(773, 600),
            nn.ReLU(),
            nn.Linear(600, 400),
            nn.ReLU(),
            nn.Linear(400, 200),
            nn.ReLU(),
            nn.Linear(200, 100)
        )
        self.encoder[0].weight.data = self.weights1
        self.encoder[2].weight.data = self.weights2
        self.encoder[4].weight.data = self.weights3
        self.encoder[6].weight.data = self.weights4
    
    def forward(self, x):
        return self.encoder(x)

In [12]:
class DeepChess(nn.Module):
    def __init__(self):
        super(DeepChess, self).__init__()
        self.pos2vec1 = Pos2Vec()
        self.pos2vec2 = Pos2Vec()
        self.fc_layers = nn.Sequential(
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 2)
        )
    
    def forward(self, x1, x2):
        x1 = self.pos2vec1(x1)
        x2 = self.pos2vec2(x2)
        x = torch.cat((x1, x2), dim=1)
        x = self.fc_layers(x)
        return torch.softmax(x,dim=1)

# Training

In [ ]:
class ChessPositionDataset(Dataset):
    def __init__(self, W_win, B_win, W_label, B_label):
        self.W_win = W_win
        self.B_win = B_win
        self.W_label = W_label
        self.B_label = B_label
    
    def __len__(self):
        return len(self.W_win)
    
    def __getitem__(self, index):
        return self.W_win[index], self.B_win[index], self.W_label[index], self.B_label[index]

In [80]:
White_win=W_win[20000:40000]
Black_win=B_win[20000:40000]
White_label=W_label[20000:40000]
Black_label=B_label[20000:40000]
for i in range(len(White_win)//2):
    White_win[2*i],Black_win[2*i]=Black_win[2*i],White_win[2*i]
    White_label[2*i],Black_label[2*i]=Black_label[2*i],White_label[2*i]

In [92]:
White_train, White_test,White_label_train ,White_label_test = train_test_split(White_win,
                                                                               White_label,
                                                                               test_size=0.1,shuffle=False)
Black_train, Black_test,Black_label_train ,Black_label_test = train_test_split(Black_win,                                       
                                                                               Black_label,
                                                                               test_size=0.1,shuffle=False)

In [93]:
White_train=torch.tensor(White_train,dtype=torch.float32)
Black_train=torch.tensor(Black_train,dtype=torch.float32)
White_label_train=torch.tensor(White_label_train,dtype=torch.float32)
Black_label_train=torch.tensor(Black_label_train,dtype=torch.float32)

In [94]:
White_test=torch.tensor(White_test,dtype=torch.float32)
Black_test=torch.tensor(Black_test,dtype=torch.float32)
White_label_test=torch.tensor(White_label_test,dtype=torch.float32)
Black_label_test=torch.tensor(Black_label_test,dtype=torch.float32)
Label_test=torch.cat((White_label_test,Black_label_test),dim=1)

In [95]:
Label_test

tensor([[0., 1.],
        [1., 0.],
        [0., 1.],
        ...,
        [1., 0.],
        [0., 1.],
        [1., 0.]])

In [96]:
TrainData=ChessPositionDataset(White_train,Black_train,White_label_train,Black_label_train)
TestData=ChessPositionDataset(White_test,Black_test,White_label_test,Black_label_test)

In [97]:
len(TrainData)

18000

In [98]:
train_loader = DataLoader(TrainData, batch_size=500, shuffle=True)
test_loader = DataLoader(TestData, batch_size=500)

In [99]:
model= DeepChess()
optimizer=optim.Adam(model.parameters(),lr=0.01)
criterion=nn.CrossEntropyLoss()

num_epochs=1000

for epochs in trange(num_epochs):
    model.train()
    for Win,Loss,Win_label,Loss_label in train_loader:
        optimizer.zero_grad()
        outputs=model(Win,Loss)
        Label=torch.cat((Win_label,Loss_label),dim=1)
        loss=criterion(outputs,Label)
        loss.backward()
        optimizer.step()
    if epochs%100==0:
        print(f"epochs:{epochs} loss:{loss}")

  0%|          | 0/1000 [00:00<?, ?it/s]

epochs:0 loss:0.775261640548706
epochs:100 loss:0.7952616214752197
epochs:200 loss:0.7892615795135498
epochs:300 loss:0.8192616105079651
epochs:400 loss:0.8272615671157837
epochs:500 loss:0.8172615766525269
epochs:600 loss:0.8592616319656372
epochs:700 loss:0.8032616376876831
epochs:800 loss:0.8172615766525269
epochs:900 loss:0.8272616863250732


In [100]:
loss

tensor(0.8193, grad_fn=<DivBackward1>)

In [101]:
torch.save(model,"DeepChess_model.pt")
torch.save(optimizer.state_dict,"DeepChess_optimizer.pt")

# Test